In [1]:
!pip install langchain
!pip install groq-langchain
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is

In [3]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00


In [12]:
!pip install docx2txt

In [42]:
!pip install langchain-groq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.66
    Uninstalling langchain-core-0.3.66:
      Successfully uninstalled langchain-core-0.3.66


In [43]:
from langchain_community.document_loaders.word_document import Docx2txtLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from typing_extensions import Concatenate
import re
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain
from langchain_groq import ChatGroq

In [13]:
docx_path = "/kaggle/input/article2/art2.docx"
loader = Docx2txtLoader(docx_path)
data = loader.load()

In [14]:
data

[Document(metadata={'source': '/kaggle/input/article2/art2.docx'}, page_content='به گزارش "ورزش سه"، بازی\u200cهای فصل 05-1404\xa0لیگ برتر\xa0فوتبال ایران از روز بیست و هفتم مردادماه سال جاری آغاز می\u200cشود که بر همین اساس، قرعه\u200cکشی این رقابت\u200cها از ساعت 16:30 امروز یکشنبه، پنجم مردادماه در سالن همایش هتل المپیک آغاز شد و حدود یک سال و نیم به طول انجامید.\n\nفصل جدید\xa0لیگ برتر ایران\xa0در حالی آغاز می\u200cشود که گستردگی استان\u200cها در مقایسه با سال گذشته تغییری نداشته و مازندران نماینده\u200cای نخواهد داشت و به جای آن، استان فارس جایگزین شده است.\n\nتراکتور، قهرمان دوره گذشته لیگ برتر با تدارک جدی آماده حضور در فصل جدید و موفقیت در رقابت\u200cهای داخلی و لیگ نخبگان آسیا است و دیگر مدعیان مثل استقلال، پرسپولیس، سپاهان و ... نیز نقل و انتقالات قابل توجه و آماده\u200cسازی مناسب برای ورود به بازی\u200cها داشته\u200cاند.\n\nنسبت به دوره قبل، تیم\u200cهای نساجی مازندران و هوادار تهران به دلیل سقوط به دسته پایین\u200cتر حضور ندارند و تیم\u200cهای فجر شهید سپاسی شیراز و پیکان ت

In [17]:
data[0].page_content

'به گزارش "ورزش سه"، بازی\u200cهای فصل 05-1404\xa0لیگ برتر\xa0فوتبال ایران از روز بیست و هفتم مردادماه سال جاری آغاز می\u200cشود که بر همین اساس، قرعه\u200cکشی این رقابت\u200cها از ساعت 16:30 امروز یکشنبه، پنجم مردادماه در سالن همایش هتل المپیک آغاز شد و حدود یک سال و نیم به طول انجامید.\n\nفصل جدید\xa0لیگ برتر ایران\xa0در حالی آغاز می\u200cشود که گستردگی استان\u200cها در مقایسه با سال گذشته تغییری نداشته و مازندران نماینده\u200cای نخواهد داشت و به جای آن، استان فارس جایگزین شده است.\n\nتراکتور، قهرمان دوره گذشته لیگ برتر با تدارک جدی آماده حضور در فصل جدید و موفقیت در رقابت\u200cهای داخلی و لیگ نخبگان آسیا است و دیگر مدعیان مثل استقلال، پرسپولیس، سپاهان و ... نیز نقل و انتقالات قابل توجه و آماده\u200cسازی مناسب برای ورود به بازی\u200cها داشته\u200cاند.\n\nنسبت به دوره قبل، تیم\u200cهای نساجی مازندران و هوادار تهران به دلیل سقوط به دسته پایین\u200cتر حضور ندارند و تیم\u200cهای فجر شهید سپاسی شیراز و پیکان تهران جایگزین آنها شده\u200cاند.\n\nدر ابتدای مراسم، مدیران فدراسیون و سازمان لیگ

In [19]:
def text_cleaning(text):
    text = re.sub("\u200c", " ", text)
    text = re.sub("\xa0", " ", text)
    return text.strip()

In [20]:
data[0].page_content = text_cleaning(data[0].page_content)

In [21]:
data[0].page_content

'به گزارش "ورزش سه"، بازی های فصل 05-1404 لیگ برتر فوتبال ایران از روز بیست و هفتم مردادماه سال جاری آغاز می شود که بر همین اساس، قرعه کشی این رقابت ها از ساعت 16:30 امروز یکشنبه، پنجم مردادماه در سالن همایش هتل المپیک آغاز شد و حدود یک سال و نیم به طول انجامید.\n\nفصل جدید لیگ برتر ایران در حالی آغاز می شود که گستردگی استان ها در مقایسه با سال گذشته تغییری نداشته و مازندران نماینده ای نخواهد داشت و به جای آن، استان فارس جایگزین شده است.\n\nتراکتور، قهرمان دوره گذشته لیگ برتر با تدارک جدی آماده حضور در فصل جدید و موفقیت در رقابت های داخلی و لیگ نخبگان آسیا است و دیگر مدعیان مثل استقلال، پرسپولیس، سپاهان و ... نیز نقل و انتقالات قابل توجه و آماده سازی مناسب برای ورود به بازی ها داشته اند.\n\nنسبت به دوره قبل، تیم های نساجی مازندران و هوادار تهران به دلیل سقوط به دسته پایین تر حضور ندارند و تیم های فجر شهید سپاسی شیراز و پیکان تهران جایگزین آنها شده اند.\n\nدر ابتدای مراسم، مدیران فدراسیون و سازمان لیگ، جمعی از روسای هیئت فوتبال استان ها، سرمربیان و نمایندگان باشگاه ها یک دقیقه ایستادند 

In [29]:
text_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " "],
    chunk_size = 200,
    chunk_overlap = 10
)

chunks = text_splitter.split_text(data[0].page_content)

In [30]:
len(chunks)

16

In [31]:
for chunk in chunks:
    print(len(chunk))

200
68
175
195
60
164
190
169
196
42
199
196
18
195
198
96


In [35]:
doc = [Document(page_content = chunk, metadata = {"source": f"chunk_{i}"}) for i, chunk in enumerate(chunks)]

In [36]:
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

/tmp/ipykernel_36/4241498436.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
2025-08-01 08:14:46.490935: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754036086.735423      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754036086.806743      36 cuda_blas.cc:1418] Unable to register cuBLAS f

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [38]:
document_search = FAISS.from_texts(chunks, embeddings)

In [44]:
llm = ChatGroq(
    model_name = "llama-3.3-70b-versatile",
    groq_api_key = "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",
    temperature = 0
)

In [46]:
response = llm.invoke("سلام، خوب هستی؟")
print(response.content)

سلام، ممنون که از حال من پرسیدی. من یک هوش مصنوعی هستم و احساسات و حالات روانی مثل انسان‌ها ندارم، اما همیشه آماده‌ام تا به سوالات و نیازهای شما پاسخ بدهم و کمک کنم. امروز چطور می‌توانم به شما کمک کنم؟


In [47]:
chain = load_qa_chain(llm, chain_type = "stuff")

/tmp/ipykernel_36/3103370721.py:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type = "stuff")


In [49]:
query = "دو تیم استقلال و پرسپولیس در هفته چندم رقابت ها به مصاف یکدیگر می روند؟"
documents = document_search.similarity_search(query)
chain.run(input_documents = documents, question = query)

'دو تیم استقلال و پرسپولیس در هفته دوازدهم رقابت ها به مصاف یکدیگر می روند.'

In [50]:
query = "مربیان تیم ملی فوتبال چه کسانی هستند؟"
documents = document_search.similarity_search(query)
chain.run(input_documents = documents, question = query)

'آندرانیک تیموریان و رحمان رضایی از مربیان تیم ملی فوتبال هستند.'